# Decoding CTC output

In [4]:
import pickle
import torch


# Load precomputed CTC output
with open('mystery_records.pickle', 'rb') as f:
    batch = pickle.load(f)

# log probabilities of softmax layers [batch_size, T, vocab_size]
log_probs = batch["log_probs"]

# Dictionary with index to character mapping
ind2char = batch["ind2char"]

# Index of special EMPTY token
EMPTY_TOK = '^'
EMPTY_IND = 0

In [6]:
log_probs.size()

torch.Size([2, 655, 28])

In [8]:
ind2char

{0: '^',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 27: ' '}

In [9]:
rec.argmax(-1).numpy()

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       23, 23,  0,  5,  0,  0,  0,  0,  0,  0, 27, 14, 14, 15,  0,  0,  0,
        0, 19, 19,  0, 20, 20,  0, 18,  0,  0, 14, 14,  0,  0,  7,  7,  0,
        5,  0,  0,  0,  0, 19, 19,  0,  0, 20, 20,  0, 15,  0,  0,  0,  0,
        0,  0, 27, 27, 12, 12,  0, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 22, 22,  5,  5,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0, 27, 27, 27, 25, 25, 15, 21, 21,  0, 27, 27, 11, 11, 14, 15,
        0,  0, 23,  0,  0, 27, 27, 20,  8,  5,  0,  0,  0,  0,  0,  0, 18,
        0,  0, 15,  0,  0,  0, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,
       19, 19,  0,  0,  0

In [14]:
def ctc_decode(inds, ind2char):
    decoder = [] # хранилище для нормальной, декодированной последовательности
    last_char_ind = EMPTY_IND
    for ind in inds:
        if last_char_ind == ind:
            continue
        if ind != EMPTY_IND:
            decoder.append(ind2char[ind])
        last_char_ind = ind
    return ''.join(decoder)

for i, rec in enumerate(log_probs):
    text = ctc_decode(rec.argmax(-1).numpy(), ind2char)
    print(f"{i}) {text}")



0) we nostrngesto love you know therols and so do i a foll commitment what i thinking of you wolden get this from any ather guy
1)  never gona give you up never donelet you down never go arun around and deset you never gon a make you cri never gonna say good by


# Computing WER and CER
Task: Implemet WER and CER metrics

In [27]:
# library for fast quick calculation of edit distance
import editdistance

# версия авторов
def calc_wer(target_text: str, pred_text: str):
    if len(target_text) == 0:
        return 1
    return editdistance.eval(target_text.split(), pred_text.split())/len(target_text.split())

# моя реализация WER, до того, как я посмотрел, чтобы сделать работу над ошибками
# разумеется это некорректно, т.к., возможно смещение слов и я просто все сделаю неправильно
# ошибки - лучшие учителя!)
def my_calc_wer(target_text: str, pred_text: str):
    target_text = target_text.split()
    pred_text = pred_text.split()
    counter = 0
    for i in range(len(target_text)):
        if target_text[i] != pred_text[i]:
            counter += 1
    return (counter/len(target_text))*100
    

def calc_cer(target_text: str, pred_text: str):
    if len(target_text) == 0:
        return 1

    return editdistance.eval(target_text, pred_text)/len(target_text)


In [28]:
import numpy as np

for target, pred, expected_wer, expected_cer in [
    ("if you can not measure it you can not improve it", 
     "if you can nt measure t yo can not i", 
     0.454, 0.25),
    ("if you cant describe what you are doing as a process you dont know what youre doing", 
     "if you cant describe what you are doing as a process you dont know what youre doing", 
     0.0, 0.0),
    ("one measurement is worth a thousand expert opinions", 
     "one  is worth thousand opinions", 
     0.375, 0.392)
]:
    wer = calc_wer(target, pred)
    cer = calc_cer(target, pred)
    assert np.isclose(wer, expected_wer, atol=1e-3), f"true: {target}, pred: {pred}, expected wer {expected_wer} != your wer {wer}"
    assert np.isclose(cer, expected_cer, atol=1e-3), f"true: {target}, pred: {pred}, expected cer {expected_cer} != your cer {cer}"
    

Task: come up with such a pair of target-prediction texts, so the
1) WER > 1.0
2) CER > WER

In [34]:
# 1) WER > 1.0
# your code here
target, prediction = "a" , "a a a a a "
assert calc_wer(target, prediction) > 1.0

# 2) CER > WER
# your code here
target, prediction = "a a a", "bbbbbb a a"
assert calc_wer(target, prediction) < calc_cer(target, prediction) 


# Beam search
Task: implement beam-search on CTC outputs

In [39]:
# Load precomputed CTC output
with open('lj_batch.pickle', 'rb') as f:
    batch = pickle.load(f)

# log probabilities of softmax layers [batch_size, T, vocab_size]
log_probs = batch["log_probs"]

# Dictionary with index to character mapping
ind2char = batch["ind2char"]

true_texts = batch["text"]

print(log_probs.size())
print(ind2char)
print(true_texts)

torch.Size([10, 310, 28])
{0: '^', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: ' '}
['he would go to her and tell her all his family complications', 'he did not say the last as a boast but merely as an assurance to the liveryman who he saw was anxious on his account', 'he started to conscious confusion only neither knowing where he was nor what he did', "i'm here because the matter is of utmost importance and brandd is the one i must see now stand aside", "of course it ain't said missus bozzle", 'mister verloc was fully responsive now', 'oh what shall we do for a home', "line of battle was formed on the north bank of stone's river on the yankee side", 'from fifteen to twenty minutes will be required to bake them nicely', 'whom is he going to flog now']


In [43]:
# возьмем просто prob через экспоненты
probs = log_probs.exp()
print(probs)

tensor([[[9.9979e-01, 1.5032e-06, 2.3595e-07,  ..., 3.1434e-07,
          1.0038e-08, 2.7595e-09],
         [9.9998e-01, 8.6571e-08, 9.8482e-09,  ..., 4.6133e-08,
          1.7876e-10, 7.6316e-11],
         [1.0000e+00, 2.2588e-08, 2.7001e-09,  ..., 6.8390e-09,
          1.1855e-11, 5.2869e-12],
         ...,
         [1.0000e+00, 4.8414e-08, 1.0775e-08,  ..., 4.0093e-09,
          6.8794e-12, 5.7046e-08],
         [9.9932e-01, 1.5367e-06, 3.9040e-07,  ..., 1.1833e-07,
          1.5983e-10, 6.7376e-04],
         [6.8953e-01, 4.6144e-04, 1.8930e-04,  ..., 1.8095e-04,
          2.3677e-07, 3.0813e-01]],

        [[9.9993e-01, 9.8160e-07, 4.9533e-07,  ..., 1.9043e-07,
          1.2068e-09, 1.8334e-11],
         [9.9999e-01, 9.7628e-08, 1.5065e-07,  ..., 7.9602e-08,
          6.0507e-11, 3.7560e-13],
         [1.0000e+00, 1.3857e-08, 3.5957e-08,  ..., 6.2041e-09,
          1.8368e-12, 1.6559e-14],
         ...,
         [1.0000e+00, 1.0177e-07, 3.9941e-09,  ..., 3.9199e-09,
          9.742

In [68]:
from collections import defaultdict
from tqdm import tqdm

# Функция для расширения и слияния путей
def expand_and_merge_path(dp, next_token_probs, ind2char):
    new_dp = defaultdict(float)
    for ind, next_token_prob in enumerate(next_token_probs):
        cur_char = ind2char[ind]
        for (prefix, last_char), v in dp.items():
            if last_char == cur_char:
                new_prefix = prefix  # Если последний символ тот же, не изменяем префикс
            else:
                if cur_char != EMPTY_TOK:
                    new_prefix = prefix + cur_char  # Если символ пустой, добавляем его в префикс
                else:
                    new_prefix = prefix  # Для всех других символов, добавляем их в префикс
            new_dp[(new_prefix, cur_char)] += v * next_token_prob  # Обновляем вероятности
    return new_dp

# Функция для усечения путей (beam search)
def truncate_paths(dp, beam_size):
    # Возвращаем лучшие beam_size путей
    return dict(sorted(dp.items(), key=lambda x: -x[1])[:beam_size])

# Основная функция для CTC beam search
def ctc_beam_search(probs, beam_size, ind2char):
    dp = {
        ('', EMPTY_TOK): 1.0,  # Начальный путь с пустым токеном
    }
    for prob in probs:
        dp = expand_and_merge_path(dp, prob, ind2char)  # Расширение путей
        dp = truncate_paths(dp, beam_size)  # Усечение путей до лучшего beam_size
    dp = [(prefix, proba) for (prefix, _), proba in sorted(dp.items(), key=lambda x: -x[1])]
    return dp

# Пример применения для log_probs
bs_results = []
for log_probs_line in log_probs:
    bs_results.append(ctc_beam_search(log_probs_line.exp().numpy(), 100, ind2char))

In [70]:

for i in range(len(true_texts)):
    beam_search_hypos = bs_results[i][:3]
    true_text = true_texts[i]
    argmax_text = ctc_decode(log_probs[i].numpy().argmax(-1), ind2char)
    print("True: ", true_text)
    print(f"Argmax: {argmax_text} --- (CER: {calc_cer(true_text, argmax_text):.3f})")
    for ind, (hypo, score) in enumerate(beam_search_hypos):
        print(f"{ind+1}) '{hypo}' --- (CER: {calc_cer(true_text, hypo):.3f})")
    print('-' * 100)

True:  he would go to her and tell her all his family complications
Argmax: he wld ge toher iand tell her all mhisan ly omblications --- (CER: 0.200)
1) 'he wl ge to her iand tell her all hisan ly omblications' --- (CER: 0.183)
2) 'he wl ge to her and tell her all hisan ly omblications' --- (CER: 0.167)
3) 'he wl ge to her iand tell her all hisanly omblications' --- (CER: 0.183)
----------------------------------------------------------------------------------------------------
True:  he did not say the last as a boast but merely as an assurance to the liveryman who he saw was anxious on his account
Argmax: he did not sad the last is a bost but mearlioves an asurance to the livery man who re saw was anxes on his account --- (CER: 0.129)
1) 'he did not say the last is a bost but merli oves an a surance to the livery man who re saw was anxes on his account' --- (CER: 0.112)
2) 'he did not say the last as a bost but merli oves an a surance to the livery man who re saw was anxes on his acc

У меня слишком высокий показатель ошибки. видимо это связано с тем, что учитывается ^ данный символ.